In [1]:
import numpy as np
import gymnasium as gym

# Create the FrozenLake environment (default: 4x4 grid, slippery surface)
env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True, render_mode='human')



In [2]:
# Hyperparameters
gamma = 0.99            # Discount factor
theta = 1e-8            # Convergence threshold
max_iterations = 1000   # Optional iteration limit


In [3]:
def value_iteration(env, gamma=0.99, theta=1e-8):
    """
    Perform value iteration to compute the optimal state-value function.
    """
    value_table = np.zeros(env.observation_space.n)

    for i in range(max_iterations):
        delta = 0

        for state in range(env.observation_space.n):
            old_value = value_table[state]

            action_values = []
            for action in range(env.action_space.n):
                value = 0
                for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                    value += prob * (reward + gamma * value_table[next_state] * (not done))
                action_values.append(value)

            value_table[state] = max(action_values)
            delta = max(delta, abs(old_value - value_table[state]))

        if delta < theta:
            print(f"Converged in {i+1} iterations.")
            break

    return value_table

In [4]:
def extract_policy(env, value_table, gamma=0.99):
    """
    Derive the optimal policy from the value function.
    """
    policy = np.zeros(env.observation_space.n, dtype=int)

    for state in range(env.observation_space.n):
        action_values = []

        for action in range(env.action_space.n):
            value = 0
            for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                value += prob * (reward + gamma * value_table[next_state] * (not done))
            action_values.append(value)

        policy[state] = np.argmax(action_values)

    return policy

In [5]:
# Run value iteration and extract optimal policy
optimal_value_table = value_iteration(env, gamma, theta)

Converged in 347 iterations.


In [6]:
optimal_policy = extract_policy(env, optimal_value_table, gamma)

env.close()

In [7]:
# Display results
print("\nOptimal Value Function:")
print(optimal_value_table.reshape((8,8)))



Optimal Value Function:
[[0.41464029 0.42720516 0.44614817 0.46832032 0.49244367 0.51656979
  0.53526148 0.54097518]
 [0.41168636 0.42120777 0.43749567 0.45838851 0.48324009 0.51353174
  0.54576783 0.55736838]
 [0.39675202 0.39384048 0.37549622 0.         0.42167796 0.49381917
  0.56121205 0.58585888]
 [0.36927222 0.35298248 0.30653119 0.20040369 0.30075272 0.
  0.56901586 0.62825901]
 [0.33266384 0.2913753  0.19730914 0.         0.28929024 0.36195179
  0.53481943 0.6896973 ]
 [0.30613619 0.         0.         0.08627638 0.21393258 0.27271393
  0.         0.77203551]
 [0.28888542 0.         0.05769637 0.04751101 0.         0.25052147
  0.         0.87776873]
 [0.28038877 0.20081497 0.12732648 0.         0.23959086 0.48644205
  0.7371033  0.        ]]


In [8]:
print("\nOptimal Policy (0=Left, 1=Down, 2=Right, 3=Up):")
print(optimal_policy.reshape((8,8)))


Optimal Policy (0=Left, 1=Down, 2=Right, 3=Up):
[[3 2 2 2 2 2 2 2]
 [3 3 3 3 3 2 2 1]
 [3 3 0 0 2 3 2 1]
 [3 3 3 1 0 0 2 2]
 [0 3 0 0 2 1 3 2]
 [0 0 0 1 3 0 0 2]
 [0 0 1 0 0 0 0 2]
 [0 1 0 0 1 2 1 0]]


In [9]:
# # For Colab

# def run_policy(env, policy, episodes=3):
#     """
#     Simulates the given policy in the FrozenLake environment.
#     Uses text rendering (ANSI) for each step.
#     """
#     # Re-create the environment with rendering enabled
#     env = gym.make("FrozenLake-v1", is_slippery=True, render_mode="ansi", map_name="8x8")

#     for ep in range(episodes):
#         state, info = env.reset()
#         done = False
#         total_reward = 0
#         steps = 0

#         print(f"\n--- Episode {ep + 1} ---")

#         while not done:
#             action = policy[state]
#             next_state, reward, terminated, truncated, _ = env.step(action)
#             done = terminated or truncated
#             total_reward += reward
#             steps += 1

#             # Render text output of the environment after each action
#             print(env.render())

#             state = next_state

#         print(f"Episode finished in {steps} steps with reward: {total_reward}")


# # Simulate the optimal policy
# run_policy(env, optimal_policy, episodes=3)

In [ ]:
import time

def run_policy(env, policy, episodes=3, delay=0.5):
    """
    Simulates the given policy in the FrozenLake environment.
    Uses graphical rendering (render_mode='human').
    """
    # Create a new environment with graphical rendering
    env = gym.make("FrozenLake-v1", map_name="8x8", is_slippery=True, render_mode='human')

    for ep in range(episodes):
        state, info = env.reset()
        done = False
        total_reward = 0
        steps = 0

        print(f"\n--- Episode {ep + 1} ---")

        while not done:
            action = policy[state]
            state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            steps += 1

            # # Add delay so you can visually follow each step
            # time.sleep(delay)
            
            print(f"\n Episode {ep + 1}: Step {steps}")

            # the goal position in the 4x4 map can be calculated as follows: 3 * 4 + 3 = 15.
            if reward:
                print("You Won!!!")
                print(f"Episode {ep + 1} ended with reward: {reward}")
            
            #Check for Termination
            if done:
                print("GAME OVER --- Terminated!!!")
                print(f"Episode {ep + 1} ended with reward: {reward}")
            

        print(f"Episode finished in {steps} steps with reward: {total_reward}")



    env.close()


run_policy(env, optimal_policy, episodes=3, delay=0.75)

/Users/oysterable/delete/ReinforcementLearning/rlvenv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists



--- Episode 1 ---

 Episode 1: Step 1

 Episode 1: Step 2

 Episode 1: Step 3

 Episode 1: Step 4

 Episode 1: Step 5

 Episode 1: Step 6

 Episode 1: Step 7

 Episode 1: Step 8

 Episode 1: Step 9

 Episode 1: Step 10

 Episode 1: Step 11

 Episode 1: Step 12

 Episode 1: Step 13

 Episode 1: Step 14

 Episode 1: Step 15

 Episode 1: Step 16

 Episode 1: Step 17

 Episode 1: Step 18

 Episode 1: Step 19

 Episode 1: Step 20

 Episode 1: Step 21

 Episode 1: Step 22

 Episode 1: Step 23

 Episode 1: Step 24

 Episode 1: Step 25

 Episode 1: Step 26

 Episode 1: Step 27

 Episode 1: Step 28

 Episode 1: Step 29

 Episode 1: Step 30

 Episode 1: Step 31

 Episode 1: Step 32

 Episode 1: Step 33

 Episode 1: Step 34

 Episode 1: Step 35

 Episode 1: Step 36

 Episode 1: Step 37

 Episode 1: Step 38

 Episode 1: Step 39

 Episode 1: Step 40

 Episode 1: Step 41

 Episode 1: Step 42

 Episode 1: Step 43

 Episode 1: Step 44

 Episode 1: Step 45

 Episode 1: Step 46

 Episode 1: Step 47

 E

: 